In [ ]:
import os
import numpy as np
import tensorflow as tf
print(tf.__version__)
from scipy.io import loadmat
import h5py
import functools

# %matplotlib widget
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update({'font.size': 16})
mpl.rcParams.update({'font.family': 'serif'})

from utils_data import read_tfrecord_dataset

TEST_NAME = 'p4677'
DATA_DIR = '/misc/projects/criticalstress-ldrd/cwj/public/Lab_AE/data_lab_p4677/'

In [ ]:
if not os.path.exists('./Data/'):
  os.makedirs('./Data/')

# Mech data
with h5py.File(DATA_DIR+TEST_NAME+'_mechdata.hdf5', 'r') as f:
  time_mu = f.get('time')[:].astype(np.float64)
  mu = f.get('mu')[:].astype(np.float32)

# AE data info
Syncfile = loadmat(DATA_DIR+'p4677_AcTimeVector_Run2.mat')
acsettings = loadmat(DATA_DIR+'p4677_run2_AE.mat')

acTime = Syncfile['acTime'][0,:]
WFlength = int(acsettings['Nsamples'][0])
acTime_dt = np.mean(acTime[1:]-acTime[:-1])/WFlength
fs = 1.0/acTime_dt

# parameters data
params_data = {}
params_data['sample_freq']        = 1e3
params_data['ae_sample_freq']     = fs
params_data['tfrecord_data_dir']  = DATA_DIR+TEST_NAME+'_data_tfrecord/'
params_data['numCHR']             = len(acsettings['channels2save'][0])

# AE dataset
wf_ae_dataset = read_tfrecord_dataset(params_data)

In [ ]:
from scipy import signal

time_ae = []
wf_ae = []
ii = 0
for ds in wf_ae_dataset:
    if ii%100000==0: print(ii)
    
    wf_ae_i = ds['wf_ae'][0,:,:].numpy()
    wf_ae_i = np.mean(wf_ae_i, axis=-1, keepdims=True)
    wf_ae_i = signal.resample(wf_ae_i, num=4096, axis=0)
    wf_ae_i = signal.decimate(wf_ae_i, 4, axis=0)
    wf_ae_i = signal.decimate(wf_ae_i, 2, axis=0)
    wf_ae.append(np.expand_dims(wf_ae_i, axis=0))
    
    t_win_i = ds['t_win'][0,:].numpy()
    time_ae.append(t_win_i[0:1])
    
    ii += 1

time_ae = np.concatenate(time_ae, axis=0).astype(np.float32)
wf_ae = np.concatenate(wf_ae, axis=0).astype(np.float32)

In [ ]:
print(time_ae.shape, wf_ae.shape)
print(time_mu.shape, mu.shape)
print(np.min(wf_ae), np.max(wf_ae))

with h5py.File('./Data/p4677_seismic_data_2MPa.hdf5', 'w') as f:
  f.create_dataset('time', data = time_ae)
  f.create_dataset('wf_ae', data = wf_ae)

with h5py.File('./Data/p4677_shear_data_2MPa.hdf5', 'w') as f:
  f.create_dataset('time', data = time_mu)
  f.create_dataset('mu', data = mu)